# 单历元定位要求基于GPS系统的C1C和C2W观测值，组成无电离层组合的观测值，计算2019年7月15号06:55:30的坐标。
## hkws1960.19o是观测文件，igs20621.sp3是精密星历文件。

##### 1.读取hkws1960.19o观测文件，将GPS系统的C1C和C2W观测值分别存在两个列表中。

In [1]:
import numpy as np

obsfile = 'hkws1960.19o'
satnames = [] # 保存卫星编号
c1c = [] # 保存L1伪距观测值
c2w = [] # 保存L2伪距观测值

with open(obsfile) as file_object:
    for line in file_object:
        if line[2:29] == '2019 07 15 06 55 30.0000000':
            number = int(line[32:35])
            for i in range(number):
                if line[0] == 'G':  # 选出所有GPS观测数据
                    # 每个数据占16列，第i个数据的起始列数为4 + 16(𝑖 − 1)
                    satnames.append(line[0:3])
                    c1c.append(float(line[3+16*(1-1):4+16*(1-1)+16]))  # C1C为第1个数据
                    c2w.append(float(line[3+16*(9-1)-1:4+16*(9-1)+16]))  #C2W为第9个数据
                line = file_object.readline() # 读取行不以字符'G'开头，继续向下读取
            break # 收集完'2019 07 15 06 55 30.0000000'时的数据即可退出文件读取循环

if len(c1c)==len(c2w):
    satnum = len(c1c) 
    
p1 = np.array(c1c) #存在1*9的矩阵中
p2 = np.array(c2w)

In [2]:
satnames

['G02', 'G05', 'G06', 'G09', 'G12', 'G17', 'G19', 'G23', 'G28']

##### 2.读取igs20621.sp3精密星历文件，获得卫星所在星历的时间、位置和钟差信息

In [3]:
igsfile = 'igs20621.sp3'
# 需定位的时间为2019年7月15号06:55:30 
# 根据SP3文件每15min一组数据（起始时刻为00:00:00），在需定位时间之前最接近的星历时间为2019年7月15号06:45:00 
# 从这个时间点再向前推4组，即选取2019年7月15号05:45：00 
igsT = [] # 存储观测卫星在7月15各个所取星历的时间 (min)
sXdata = [] # 存储观测卫星在各个所取星历时间的x坐标 (km)
sYdata = [] # 存储观测卫星在各个所取星历时间的y坐标 (km)
sZdata = [] # 存储观测卫星在各个所取星历时间的z坐标 (km)
sClock = [] # 存储观测卫星在各个所取星历时间的钟差(μs)

with open(igsfile) as file_object:
    for line in file_object:
        if line[3:31] == '2019  7 15  5 45  0.00000000':
            igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
            
            # 卫星位置（km）和钟差（μs）依次为X、Y、Z、钟差每个数据占14列
            # 第i个数据的起始列为 5 + 14(i - 1)
            while(len(igsT)<=10):   # 以时间t为准，向前找5组精密星历，向后找5组精密星历，需10组
                line = file_object.readline()
                for n in range(satnum):  # 读一组选取时间点的星历，存储9个观测卫星参数
                    while line[1:4] != satnames[n]:
                        line = file_object.readline()
                    sXdata.append(float(line[4+14*(1-1):5+14*(1-1)+14]))                            
                    sYdata.append(float(line[4+14*(2-1):5+14*(2-1)+14]))
                    sZdata.append(float(line[4+14*(3-1):5+14*(3-1)+14]))
                    sClock.append(float(line[4+14*(4-1):5+14*(4-1)+14]))
                if len(igsT) == 10:
                    break
                else:
                    while(line[0] != '*'):
                        line = file_object.readline()
                    igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
            break



In [4]:
igsT

[345.0, 360.0, 375.0, 390.0, 405.0, 420.0, 435.0, 450.0, 465.0, 480.0]

In [5]:
sXmtr = np.array(sXdata).reshape(10, satnum)
sYmtr = np.array(sYdata).reshape(10, satnum)
sZmtr = np.array(sZdata).reshape(10,satnum)
sCmtr = np.array(sClock).reshape(10,satnum)


In [6]:
sCmtr[0]
igsT

[345.0, 360.0, 375.0, 390.0, 405.0, 420.0, 435.0, 450.0, 465.0, 480.0]

##### 3.以igsT时间（观测卫星在7月15各个所取星历的时间）为自变量使用9阶拉格朗日插值，解算出所求时刻的观测卫星信息

In [7]:
from scipy.interpolate import lagrange

def interpolation(rho):
    xs = []
    ys = []
    zs = []
    dts = []
    t = 6*60 + 55 + 30/60   # t0为初始插值时间
    c = 299792458
    # 对于9个卫星，分别使用10个历元的卫星位置和钟差求出拉格朗日插值多项式
    # 将插值时间 t-rho/c/60代入拉格朗日插值多项式
    # 得出每个卫星的位置在时间t的位置和钟差
    for i in range(satnum):
        sx_poly = lagrange(igsT, sXmtr[:,i])
        xs.append(sx_poly(t-rho / c / 60))
        sy_poly = lagrange(igsT, sYmtr[:,i])   
        ys.append(sy_poly(t- rho / c / 60))
        sz_poly = lagrange(igsT, sZmtr[:,i])
        zs.append(sz_poly(t- rho / c / 60))
        sc_poly = lagrange(igsT, sCmtr[:,i])
        dts.append(sc_poly(t- rho / c / 60))
     # 将观测卫星的位置和钟差转换为矩阵
    satPos = np.array([xs, ys, zs])
    dTs = np.array(dts)
    satPos *= 1000 # km转换为m
    dTs *= 10**(-6) # μs转换为s
    return (satPos, dTs)

interpolation(0)

(array([[  8272674.04194651,   3004769.92813344,  -6722885.74079246,
         -24619532.42704511,  10725289.08825757, -19461235.04373543,
         -15215274.3616507 , -20044522.80090068, -12434224.86064616],
        [ 18950734.94559107,  25719521.16120499,  13377785.48364868,
           6951921.7413956 ,  15078474.80341369,  15406844.35854852,
          16040511.80308064,  -3911548.71314507,  18272442.65170731],
        [ 17361958.82564647,  -5527210.67366975,  21976650.07236875,
           7112125.88797633,  18899702.07955877,   9732189.52254548,
          14454116.67377008,  17529701.44389936, -14197818.86833319]]),
 array([-2.60448891e-04,  6.62976137e-08,  1.49128599e-04,  3.70602640e-04,
         2.23773109e-04,  8.89895810e-05, -2.93916237e-04, -1.77132049e-04,
         7.66443274e-04]))

In [8]:
sPos, T = interpolation(0)
sPos

array([[  8272674.04194651,   3004769.92813344,  -6722885.74079246,
        -24619532.42704511,  10725289.08825757, -19461235.04373543,
        -15215274.3616507 , -20044522.80090068, -12434224.86064616],
       [ 18950734.94559107,  25719521.16120499,  13377785.48364868,
          6951921.7413956 ,  15078474.80341369,  15406844.35854852,
         16040511.80308064,  -3911548.71314507,  18272442.65170731],
       [ 17361958.82564647,  -5527210.67366975,  21976650.07236875,
          7112125.88797633,  18899702.07955877,   9732189.52254548,
         14454116.67377008,  17529701.44389936, -14197818.86833319]])

In [9]:
f1 = 1575.42 # C1C载波频率
f2 = 1227.6 # C2W载波频率
PC = f1**2/(f1**2 - f2**2)*p1 - f2**2/(f1**2 - f2**2)*p2
PC

array([22928040.20791756, 22507455.08689565, 21518295.39909832,
       22625415.36830399, 23147862.10920567, 21050340.28902534,
       20631705.07151567, 25048028.2818291 , 23063324.47256333])

##### 4.使用最小二乘法求接收机x,y,z和dx,dy,dz,ddt。组成观测方程(V=L-A*dX, P)，P可取单位阵。


In [ ]:
from math import sqrt
from numpy.linalg import inv

f1 = 1575.42 # C1C载波频率
f2 = 1227.6 # C2W载波频率
c = 299792458
PC = f1**2/(f1**2 - f2**2)*p1 - f2**2/(f1**2 - f2**2)*p2 # 无电离层组合观测值
dX = np.ones(4)
X = np.zeros(4)
rex = []
redx = []
sPos, sdT = interpolation(0)

while sqrt(dX[0]**2+dX[1]**2+dX[2]**2) >= 0.0001:
    Li = []
    Ai = []
    for i in range(satnum):
        rho = sqrt((sPos[0,i]-X[0])**2+(sPos[1,i]-X[1])**2+(sPos[2,i]-X[2])**2)
        sPos, sdT = interpolation(rho)
        rho = sqrt((sPos[0,i]-X[0])**2+(sPos[1,i]-X[1])**2+(sPos[2,i]-X[2])**2)
        Li.append(PC[i]-rho-c*X[3]+c*sdT[i])
        Ai.append([(X[0]-sPos[0,i])/rho, (X[1]-sPos[1,i])/rho, (X[2]-sPos[2,i])/rho, c])

    A = np.array(Ai).reshape(satnum, 4) # 系数矩阵
    L = np.array(Li)  # 观测向量
    Q = inv(A.T.dot(A))
    dX = Q.dot(A.T.dot(L)) 
    redx.append(dX)
    X += dX
    rex.append(X)
    

In [11]:
dX

array([ 6.04820450e-09, -1.90777047e-08, -6.74626557e-09, -4.25948005e-17])

In [12]:
X

array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07])

In [13]:
rex

[array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07]),
 array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07]),
 array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07]),
 array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07]),
 array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07]),
 array([-2.42990928e+06,  5.37378955e+06,  2.41907143e+06, -7.26668017e-07])]

In [14]:
redx

[array([-2.89212542e+06,  6.29455047e+06,  2.87195627e+06,  4.08209035e-03]),
 array([ 4.49590177e+05, -8.99472521e+05, -4.41168060e+05, -3.97262637e-03]),
 array([ 1.26169301e+04, -2.12742428e+04, -1.17085077e+04, -1.10121106e-04]),
 array([ 9.02698825e+00, -1.41594629e+01, -8.27193176e+00, -6.95351872e-08]),
 array([-2.20528302e-05, -3.12093324e-04, -1.99910473e-04, -6.25817413e-13]),
 array([ 6.04820450e-09, -1.90777047e-08, -6.74626557e-09, -4.25948005e-17])]